In [1]:
import random
import string
import hdbscan
import numpy as np
import pandas as pd
from sgt import SGT
from faker import Faker

In [18]:
fake = Faker()
emails_list = []
larry_emails = {}
benny_emails = {}
for i in range(100):
    base_string = 'larry'
    larry_emails[i] = base_string + str(i) + '@gmail.com'
    base_string = 'benny'
    benny_emails[i] = base_string + str(i) + '@gmail.com'
    
for i in range(1000):
    emails_list.append(f"{fake.first_name_female()}.{fake.last_name()}@{fake.domain_name()}")
    emails_list.append(f"{fake.first_name_male()}.{fake.last_name()}@{fake.domain_name()}")

In [19]:
emails = pd.DataFrame(emails_list, columns=['email'])
emails = emails.reset_index()
emails.columns=['id', 'email']

larry_emails = pd.DataFrame(larry_emails, index=[1]).T
larry_emails.columns=['email']
larry_emails['id'] = 2050
larry_emails = larry_emails.reset_index(drop=True)

benny_emails = pd.DataFrame(benny_emails, index=[1]).T
benny_emails.columns=['email']
benny_emails['id'] = 2939
benny_emails = benny_emails.reset_index(drop=True)

In [20]:
corpus = pd.concat([benny_emails, larry_emails])
corpus = pd.concat([corpus, emails])
corpus[['sequence', 'domain']] = corpus['email'].str.split('@', expand=True)
corpus['sequence'] = corpus['sequence'].str.replace('[^a-zA-Z]+', '')
corpus['sequence'] = corpus['sequence'].str.lower()
corpus['sequence'] = corpus['sequence'].astype(str).map(list)
corpus = corpus.reset_index(drop=True)

In [21]:
sgt_ = SGT(kappa=5, lengthsensitive=True)
embedding_df = sgt_.fit_transform(corpus[['id', 'sequence']])
embedding_df = embedding_df.set_index('id')
embedding_df = embedding_df.fillna(0)

In [22]:
clusterer = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
                            gen_min_span_tree=False, leaf_size=40,
                            metric='euclidean', min_cluster_size=15, min_samples=None, p=None)
predictions = clusterer.fit_predict(embedding_df)
cluster_map = pd.DataFrame()
cluster_map['data_index'] = corpus.index.values
cluster_map['cluster'] = predictions

In [23]:
cluster_map.cluster.value_counts()

cluster
-1    1985
 1     100
 2     100
 0      15
Name: count, dtype: int64

In [25]:
cluster_map = cluster_map.join(corpus)

In [42]:
cluster_map[cluster_map.cluster == 0][['cluster', 'email']].sample(5)

,cluster,email
959,0,Michael.Terry@henry-brown.com
2077,0,Michael.Cox@ward.com
1565,0,Michael.Wise@shah.com
674,0,Rachael.Chan@walters.org
1833,0,Michael.Heath@davis.com


In [38]:
cluster_map[cluster_map.cluster == 1][['cluster', 'email']].sample(5)

,cluster,email
66,1,benny66@gmail.com
19,1,benny19@gmail.com
24,1,benny24@gmail.com
69,1,benny69@gmail.com
21,1,benny21@gmail.com


In [39]:
cluster_map[cluster_map.cluster == 2][['cluster', 'email']].sample(5)

,cluster,email
110,2,larry10@gmail.com
167,2,larry67@gmail.com
188,2,larry88@gmail.com
140,2,larry40@gmail.com
152,2,larry52@gmail.com


In [33]:
cluster_map[cluster_map.cluster == 3]

,data_index,cluster,email,id,sequence,domain
